#  Activity 3.0: Vectorization

This mini activity is designed to help you get more comfortable with vectorization.  There are 3 total questions in two parts.

**Due date**: Thursday October 5th, 2023, 9:00 pm EDT.

**Instructions for Submission**: Submit via Gradescope.

## Part 1: Reading Vectorized Code

In this problem you need to read, understand and explain what is going on in a few snippets of code that the compiler generated for the problem we discussed in class.  You will want to look up each instruction to understand what it is doing and what the cost is.

The code and assembly can be seen [https://godbolt.org/z/d3oKW4K3E](https://godbolt.org/z/d3oKW4K3E)

### Question 1
We first look at the loop body 

```
.LBB0_6:
        vpcmpeqd        xmm3, xmm1, xmmword ptr [rdi + 4*rax]
        vpmovzxdq       ymm3, xmm3              # ymm3 = xmm3[0],zero,xmm3[1],zero,xmm3[2],zero,xmm3[3],zero
        vpand   ymm3, ymm3, ymm2
        vpaddq  ymm0, ymm0, ymm3
        add     rax, 4
        cmp     rcx, rax
        jne     .LBB0_6
```

Please explain what this is doing, and how many cycles each iteration of the loop takes.  

Your explanation should include what are the inputs (what values are in each register at the beginning), what are the outputs (the values of the registers at the end), and how it is computing this.  Your answer should be complete in that all 7 instructions must be explained.


### Answer 1

Based on the given C++ code and its corresponding low-level assembly instructions, here is what I understood:

1. `Label .LBB0_6`: This is a loop label. The code that follows is looped until a certain condition is met.

2. `vpcmpeqd xmm3, xmm1, xmmword ptr [rdi + 4*rax]`: This instruction compares packed doubleword integers in xmm1 to those in the memory location pointed to by [rdi + 4*rax]. The result is stored in xmm3, with each element being set to all ones if the corresponding doublewords are equal, or to zero if they are not. Here, rdi points to the data array from the C++ code, and rax is used as an offset to access the data array in chunks. This is a SIMD compare instruction (which could also be executed in parallel), and should take 1-3 cycles.

3. `vpmovzxdq ymm3, xmm3`: This instruction zero-extends packed doubleword integers from xmm3 to packed quadword integers in ymm3. The comment gives a clear visualization of the transformation. This is a SIMD data movement instruction which should take 1 to 2 cycles.

4. `vpand ymm3, ymm3, ymm2`: This instruction performs a bitwise AND operation between the values in ymm3 and ymm2, storing the result in ymm3. Since ymm2 has been previously set to [1,1,1,1] (from an earlier instruction), this operation will retain values in ymm3 that are equal to 1 and set others to zero. This is a SIMD bitwise operation, which is generally quite fast, typically taking 1 cycle.

5. `vpaddq ymm0, ymm0, ymm3`: This instruction adds packed quadword integers from ymm0 to those in ymm3 and stores the result in ymm0. Essentially, this accumulates the count of target integer matches in ymm0. This is a SIMD addition operation, which usually takes around 1 to 3 cycles.

6. `add rax, 4`: This instruction increments the value in the rax register by 4, effectively moving the offset to access the next chunk of the data array. This is a basic instruction which takes 1 cycle.

7. `cmp rcx, rax`: This instruction compares the values in rcx and rax. rcx contains the rounded-down value of n (from the C++ code) to the nearest multiple of 4. The purpose of this comparison is to check whether we have processed all chunks of the data array. This is a basic comparison instruction which takes 1 cycle.

8. `jne .LBB0_6`: This instruction jumps back to the .LBB0_6 label if the values in rcx and rax are not equal, meaning there are more chunks in the data array to process. This is a branching instruction and if the branch is predicted correctly by the branch predictor, the cost can be as low as 1 cycle.

The loop can take a varied amount of time depending on the CPU's architecture but it should take a minimum of around 7 cycles.

### Question 2

After the loop

```
        vextracti128    xmm1, ymm0, 1
        vpaddq  xmm0, xmm0, xmm1
        vpshufd xmm1, xmm0, 238                 # xmm1 = xmm0[2,3,2,3]
        vpaddq  xmm0, xmm0, xmm1
        vmovq   rax, xmm0
```

Please explain what this is doing, and how many cycles it takes.  

Once again please include the inputs, outputs, and cost.  You must include an explanation for every instruction and what it is doing.

### Answer 2

Based on the previous answer, here is what I understand:

1. `vextracti128 xmm1, ymm0, 1`: From our previous answer, we know that ymm0 has been accumulating counts. This instruction extracts the upper 128 bits of the YMM register ymm0 and stores them in xmm1. This operation essentially divides ymm0 into two halves. This operation should take 1 to 3 cycles.

2. `vpaddq xmm0, xmm0, xmm1`: The counts from the two halves of ymm0 are added together and stored back in the lower half of ymm0. Now, xmm0 contains the combined counts from the entire data array processed so far. This is an SIMD addition operation, which should take around 1 to 3 cycles as well.

3. `vpshufd xmm1, xmm0, 238`: The shuffle operation rearranges the doublewords in xmm0 to get the order xmm1 = xmm0[2,3,2,3]. This effectively prepares the values in xmm0 for another addition operation. This should take around 1 to 2 cycles, especially on Intel-based architectures.

4. `vpaddq xmm0, xmm0, xmm1`: The contents of xmm0 and xmm1 are added together. At this point, the result in xmm0 is essentially the total count of integers in the data array that match the target value, accumulated across all SIMD lanes. This is another SIMD addition operation, so it should take around 1 to 3 cycles.

5. `vmovq rax, xmm0`: The final count is moved from xmm0 to the general-purpose register rax. In the x86-64 calling convention, the rax register is used to return integer values from functions. Therefore, this instruction prepares the return value of the function. This is a data movement operation, taking around 1 to 2 cycles.

Depending on the specific microarchitecture, these instructions should together take around 5-13 cycles. 

## Part 2: Writing Vectorized Code

In this part you will tackle a new problem, write some code for it, and then analyze it.  The problem can be found at [http://preview.speedcode.org/ide/index.html?count_pairs](http://preview.speedcode.org/ide/index.html?count_pairs)

The goal of the problem us to count unaligned pairs of bytes in an array.

The starting code is 
```c
uint64_t count_pairs(uint8_t *data, uint64_t size, uint8_t target) {
  uint64_t total = 0;
  for (uint64_t i = 0; i < size - 1; i++) {
    if (data[i] == target && data[i + 1] == target) {
      total += 1;
    }
  }
  return total;
}
```




### Question 3

Please achieve 1,000% speedup or more over the reference code and include your code in your submission.

You must explain your solution in English as well.  Submissions without a full explanation will not receive points. 

If you did it using intrinsics then explain your inner loop as you did for the previous problem.  
Including: 
 - how does it compute the answer?
 - how many cycles does it take?
 - how many iterations of the base loop from the starting code does it compute on each iteration?

If you did it without using intrinsics please explain what you did to transform the problem into a form that the compiler could vectorize.

Yes, this is a hint that it can be done either with, or without intrinsics


### Answer 3

Here is the code I made:

```

uint64_t count_pairs(uint8_t *data, uint64_t size, uint8_t target) {
  cilk::opadd_reducer<uint64_t> total = 0;
  cilk_for (uint64_t i = 0; i < size - 1; i++) {
    if (data[i] == target && data[i + 1] == target) {
      total += 1;
    }
  }
  return total;
}

```

I figured the compiler could vectorize by using a reduction variable, which is a variable that is accumulated across multiple iterations of a loop. In our case, the `total` variable is a reduction variable.

The compiler can vectorize the loop because it can see that the reduction variable is only used to accumulate the results of the loop iterations. This means that the loop iterations can be executed in parallel, without affecting the correctness of the program.